# Part 1 -- Setup & Collection

Pull Tweets from the Twitter API and collect **all Tweets from 30 tech thought leaders and news outlets**.

### Load lib codes:

In [2]:
!pwd

/home/jovyan/work/Github/predicting_stock_market_trends_with_Twitter


In [4]:
from os import chdir
chdir('/home/jovyan/work/Github/predicting_stock_market_trends_with_Twitter')

from lib import *

### Create database with PyMongo:

In [1]:
!pip install pymongo twitter

    100% |████████████████████████████████| 368kB 1.5MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 3.3MB/s ta 0:00:01
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
import pymongo
cli = pymongo.MongoClient(host='35.163.253.8')

In [13]:
cli.database_names()

['admin', 'local']

In [14]:
cli.drop_database('twitter_db')

In [15]:
# Create a reference. This is not going to instantiate until you put data in it:
twitter_db = cli.twitter_db
cli.database_names()

['admin', 'local']

In [16]:
twitter_collection = cli.twitter_db.twitter_collection
cli.database_names()

['admin', 'local']

### Pull data from Yelp API:

In [ ]:
CONSUMER_KEY = None
CONSUMER_SECRET = None
ACCESS_TOKEN = None
ACCESS_SECRET = None